# Using AI21 Paraphrase on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Paraphrase** using Amazon SageMaker.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Paraphrase a single sentence](#B.-Paraphrase-a-single-sentence)
   1. [Paraphrase with a ceratin style](#C.-Paraphrase-with-a-ceratin-style)
   1. [Add context to enhance the paraphrasing](#D.-Add-context-to-enhance-the-paraphrasing)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [17]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/paraphrase-1-0-005-78c72221a3a8317194bb0aecf953838b"
}

In [18]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [19]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [20]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [21]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [22]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [25]:
endpoint_name = "paraphrase"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.2xlarge"
)

### A. Create an endpoint

In [26]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Paraphrase a single sentence

**AI21 Paraphrase model** offers access to our world-class paraphrasing engine. It has been specifically developed for suggesting alternative ways to convey the same message using different words.

This model takes a piece of text and returns a list of paraphrases that convey the same meaning using different words. It contains features to enable you more control over the output by adjusting the tone and style. Moreover, you can add additional context to the text, to make the rewrite suggestion more relevant (by paraphrasing a certain range of your text).

The input text should contain **no more than 500 characters**. The output will include up to 10 suggestions to paraphrase your input text.

The following is a simple example of a paraphrase of a single sentence.

In [27]:
text = "Throughout this page, we will explore the advantages and features of the Paraphrase model."

response = ai21.Paraphrase.execute(
    text=text,
    sm_endpoint=endpoint_name
)

print(response.suggestions)

[{'text': 'In this page, we will explore the advantages and features of the Paraphrase model.'}, {'text': 'The Paraphrase model will be explored throughout this page.'}, {'text': 'The purpose of this page is to explore the features and advantages of the Paraphrase model.'}, {'text': 'The Paraphrase model is discussed throughout this page.'}, {'text': 'This page explores the advantages and features of Paraphrase.'}, {'text': 'Our goal on this page is to examine the advantages and features of the Paraphrase model.'}, {'text': "Here we'll discuss the advantages and features of the Paraphrase model."}, {'text': 'On this page, we will explore the benefits and features of the Paraphrase model.'}, {'text': 'The purpose of this page is to describe the advantages and features of the Paraphrase model.'}, {'text': 'This page will investigate the advantages and features of the Paraphrase model.'}]


Let's print the suggestions in a more comfortable way:

In [28]:
print("Original sentence:")
print(text)
print("============================")
print("Suggestions:")
print("\n".join(["- " + x['text'] for x in response.suggestions]))

Original sentence:
Throughout this page, we will explore the advantages and features of the Paraphrase model.
Suggestions:
- In this page, we will explore the advantages and features of the Paraphrase model.
- The Paraphrase model will be explored throughout this page.
- The purpose of this page is to explore the features and advantages of the Paraphrase model.
- The Paraphrase model is discussed throughout this page.
- This page explores the advantages and features of Paraphrase.
- Our goal on this page is to examine the advantages and features of the Paraphrase model.
- Here we'll discuss the advantages and features of the Paraphrase model.
- On this page, we will explore the benefits and features of the Paraphrase model.
- The purpose of this page is to describe the advantages and features of the Paraphrase model.
- This page will investigate the advantages and features of the Paraphrase model.


### C. Paraphrase with a ceratin style

You have the choice between five different styles:
- General - there are fresh and creative ways to rephrase sentences. Offer them to your users.
- Casual - convey a friendlier and more accessible tone for the right audience.
- Formal - present your words in a more professional way.
- Shorten - express your messages clearly and concisely.
- Expand - expand your sentences to give more detail, nuance and depth.

Below is an example of the same text paraphrased in differnt styles.

In [31]:
text = "This might be the best thing that has happened to my writing"

response_formal = ai21.Paraphrase.execute(
                                text=text,
                                style="formal",
                                sm_endpoint=endpoint_name
)

response_casual = ai21.Paraphrase.execute(
                                text=text,
                                style="casual",
                                sm_endpoint=endpoint_name
)

In [32]:
print("Original sentence:")
print(text)
print("=======================")
print("Formal paraphrase:")
print(response_formal.suggestions[0]['text'])
print("=======================")
print("Casual paraphrase:")
print(response_casual.suggestions[0]['text'])

Original sentence:
This might be the best thing that has happened to my writing
Formal paraphrase:
I believe this is the best thing that has ever happened to my writing
Casual paraphrase:
I think this is the best thing that's happened to my writing


### D. Add context to enhance the paraphrasing

You can paraphrase only part of the text while keeping the surrounding text unchanged by specifying a range within the text. This is done by passing to the model the range to paraphrase (by specifying _startIndex_ and _endIndex_).

**AVOID** ranges that start or end mid-word. Cases like this usually yield few or no suggestions.

See the following example for a call that combines a short style and the context feature, i.e. - paraphrase only part of the text, while maintaining the full context. The _startIndex_ can be either the character you wish to paraphrase from, or a whitespace before it.

In [33]:
full_context = "This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear. In addition to providing ultimate comfort and a timeless appearance, this t-shirt has a durable ribbed neckline and a regular fit. It is available in a variety of colors so that you can wear it up or down for any occasion. It's great for running errands or dressing up for a night out."
text_to_paraphrase = "This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear."

# Find the range to pass to the model
startIndex = full_context.find(text_to_paraphrase)
endIndex = startIndex + len(text_to_paraphrase)

response = ai21.Paraphrase.execute(text=full_context,
                                   style="short",
                                   startIndex=startIndex,
                                   endIndex=endIndex,
                                   sm_endpoint=endpoint_name
                                  )

Note that the response will always include the full text and not just the paraphrased part:

In [34]:
print("Original sentence:")
print(full_context)
print("=======================")
print("Text to paraphrase:")
print(text_to_paraphrase)
print("=======================")
print("Suggestions:")
print("\n".join(["- " + x['text'] for x in response.suggestions]))

Original sentence:
This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear. In addition to providing ultimate comfort and a timeless appearance, this t-shirt has a durable ribbed neckline and a regular fit. It is available in a variety of colors so that you can wear it up or down for any occasion. It's great for running errands or dressing up for a night out.
Text to paraphrase:
This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear.
Suggestions:
- Cotton t-shirt made from soft, comfortable cotton for everyday wear. In addition to providing ultimate comfort and a timeless appearance, this t-shirt has a durable ribbed neckline and a regular fit. It is available in a variety of colors so that you can wear it up or down for any occasion. It's great for running errands or dressing up for a night out.
- Soft, comfortable cotton makes this t-shirt perfect for everyday wear. In addition to providing ultim

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/paraphrase-api) to understand all the capabilities of AI21 Paraphrase model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [35]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [36]:
model.delete_model()